In [5]:
from ultralytics import YOLO
import cv2
import time
import torch
import matplotlib.pyplot as plt
import numpy as np 

model = YOLO("yolov8x-seg.pt")
st = time.time()

img = cv2.imread("/home/alper/Spaceport/render_process/8_02.png")
background_image = cv2.imread("/home/alper/Spaceport/render_process/background_8.png")

imgsz = 2208
kernel_size = 2
erode_iter = 5
kernel = np.ones((kernel_size, kernel_size), np.uint8)
results = model(img.copy(), imgsz=imgsz, classes=0, save=True)  # predict on an image

for result in results:
    masks = result.masks.data
    boxes = result.boxes.data
    clss = boxes[:, 5]
    people_indices = torch.where(clss == 0)
    people_masks = masks[people_indices]
    people_mask = torch.any(people_masks, dim=0).int() * 255
    mask = people_mask.cpu().numpy() # int32
    # cv2.imwrite("person_mask_output.png", mask)

mask = mask.astype(np.uint8) # 2800 x 4416
resized_mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation = cv2.INTER_LINEAR) # 3000 x 6000
processed_mask = cv2.erode(resized_mask, kernel, erode_iter)
post_processed_person_merge = cv2.bitwise_and(img, img, mask=processed_mask[:, :, None])
background_image[processed_mask != 0] = post_processed_person_merge[processed_mask != 0]

print("Consumed time -> ", time.time() - st)
cv2.imwrite(f"result_ks{kernel_size}_it_{erode_iter}_imgsize_{imgsz}.png", background_image)

(3000, 6000, 3)



0: 1120x2208 1 person, 96.5ms
Speed: 9.3ms preprocess, 96.5ms inference, 1.2ms postprocess per image at shape (1, 3, 1120, 2208)
Results saved to runs/segment/predict56


Consumed time ->  1.246295690536499


True

In [4]:
import cv2
import numpy as np

mask = cv2.imread("/home/alper/Spaceport/render_process/yolox_mask_1.png", 0)
img = cv2.imread("/home/alper/Spaceport/render_process/8_02.png")
background_image = cv2.imread("/home/alper/Spaceport/render_process/background_8.png")

kernel_size = 5
erode_iter = 5
kernel = np.ones((kernel_size, kernel_size), np.uint8)
processed_mask = cv2.erode(mask, kernel, erode_iter)
post_processed_person_merge = cv2.bitwise_and(img, img, mask=processed_mask[:, :, None])
background_image[processed_mask != 0] = post_processed_person_merge[processed_mask != 0]
cv2.imwrite(f"yolox_masked_results{kernel_size}_it_{erode_iter}.png", background_image)

True